In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
howe 17760
beltway 45721
paperwork 17953
gabrielson 156428
madan 153839
tasmania 15529
2196007
Preparing embedding matrix
Null word embeddings: 3388


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=False,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 100
for model_func,model_name in model_defines:
    for fold_k in [3,4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 10
            batch = 64
        else:
            epo = 8
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/glove_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 95s 889us/step - loss: 0.0702 - acc: 0.9772 - val_loss: 0.0480 - val_acc: 0.9820
Epoch 2/8
106380/106380 [==============================] - 94s 884us/step - loss: 0.0463 - acc: 0.9828 - val_loss: 0.0432 - val_acc: 0.9833
Epoch 3/8
106380/106380 [==============================] - 94s 884us/step - loss: 0.0424 - acc: 0.9838 - val_loss: 0.0431 - val_acc: 0.9832
Epoch 4/8
106380/106380 [==============================] - 94s 886us/step - loss: 0.0398 - acc: 0.9845 - val_loss: 0.0411 - val_acc: 0.9839
Epoch 5/8
106380/106380 [==============================] - 94s 887us/step - loss: 0.0375 - acc: 0.9854 - val_loss: 0.0408 - val_acc: 0.9841
Epoch 6/8
106380/106380 [==============================] - 97s 912us/step - loss: 0.0352 - acc: 0.9860 - val_loss: 0.0420 - val_acc: 0.9840
Epoch 7/8
106380/106380 [==============================] - 97s 914us/step - loss: 0.0323 - acc: 0.9871 - val_

Epoch 4/8
119678/119678 [==============================] - 101s 841us/step - loss: 0.0397 - acc: 0.9846 - val_loss: 0.0408 - val_acc: 0.9837
Epoch 5/8
119678/119678 [==============================] - 101s 840us/step - loss: 0.0382 - acc: 0.9849 - val_loss: 0.0405 - val_acc: 0.9841
Epoch 6/8
119678/119678 [==============================] - 101s 841us/step - loss: 0.0353 - acc: 0.9860 - val_loss: 0.0412 - val_acc: 0.9839
Epoch 7/8
119678/119678 [==============================] - 100s 840us/step - loss: 0.0327 - acc: 0.9868 - val_loss: 0.0418 - val_acc: 0.9839
Epoch 8/8
119678/119678 [==============================] - 101s 844us/step - loss: 0.0304 - acc: 0.9878 - val_loss: 0.0434 - val_acc: 0.9836
Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 104s 866us/step - loss: 0.0715 - acc: 0.9765 - val_loss: 0.0466 - val_acc: 0.9823
Epoch 2/8
119678/119678 [==============================] - 104s 871us/step - loss: 0.0452 - acc: 0.9829

Epoch 8/8
119679/119679 [==============================] - 100s 839us/step - loss: 0.0287 - acc: 0.9883 - val_loss: 0.0465 - val_acc: 0.9834
-------------------------------
0 0.0894480616499 0.965808323568
1 0.021877752778 0.990794066591
2 0.0461115303033 0.981901473325
3 0.00883202272831 0.997286474359
4 0.0590868748185 0.975772540123
5 0.0203360129592 0.993056382425
final 0.0409487092062 0.984103210065
../features/glove_cudnn_gru_4_feat.pkl done
Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 172s 2ms/step - loss: 0.0826 - acc: 0.9734 - val_loss: 0.0486 - val_acc: 0.9815
Epoch 2/8
106380/106380 [==============================] - 171s 2ms/step - loss: 0.0494 - acc: 0.9815 - val_loss: 0.0452 - val_acc: 0.9824
Epoch 3/8
106380/106380 [==============================] - 171s 2ms/step - loss: 0.0461 - acc: 0.9825 - val_loss: 0.0442 - val_acc: 0.9829
Epoch 4/8
106380/106380 [==============================] - 171s 2ms/step - loss:

5 0.0197235533133 0.992786909902
final 0.0409690700927 0.983907894709
../features/glove_lstm_v1_3_feat.pkl done
Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 180s 2ms/step - loss: 0.0762 - acc: 0.9755 - val_loss: 0.0490 - val_acc: 0.9813
Epoch 2/8
119678/119678 [==============================] - 179s 1ms/step - loss: 0.0495 - acc: 0.9817 - val_loss: 0.0474 - val_acc: 0.9819
Epoch 3/8
119678/119678 [==============================] - 179s 1ms/step - loss: 0.0467 - acc: 0.9824 - val_loss: 0.0442 - val_acc: 0.9830
Epoch 4/8
119678/119678 [==============================] - 179s 1ms/step - loss: 0.0443 - acc: 0.9830 - val_loss: 0.0428 - val_acc: 0.9834
Epoch 5/8
119678/119678 [==============================] - 179s 1ms/step - loss: 0.0431 - acc: 0.9834 - val_loss: 0.0429 - val_acc: 0.9833
Epoch 6/8
119678/119678 [==============================] - 179s 1ms/step - loss: 0.0419 - acc: 0.9838 - val_loss: 0.0421 - val_acc: 0.9833
Epo

Epoch 3/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0464 - acc: 0.9823 - val_loss: 0.0447 - val_acc: 0.9829
Epoch 4/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0442 - acc: 0.9830 - val_loss: 0.0434 - val_acc: 0.9834
Epoch 5/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0428 - acc: 0.9835 - val_loss: 0.0429 - val_acc: 0.9834
Epoch 6/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0417 - acc: 0.9838 - val_loss: 0.0419 - val_acc: 0.9839
Epoch 7/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0404 - acc: 0.9842 - val_loss: 0.0425 - val_acc: 0.9835
Epoch 8/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0393 - acc: 0.9846 - val_loss: 0.0420 - val_acc: 0.9835
-------------------------------
0 0.0895928040476 0.965494983424
1 0.0216273093203 0.990743932168
2 0.0454763901348 0.982421617963
3 0.00896593320254 0.997280207557
4

106381/106381 [==============================] - 125s 1ms/step - loss: 0.0432 - acc: 0.9834 - val_loss: 0.0470 - val_acc: 0.9825
Epoch 3/10
106381/106381 [==============================] - 125s 1ms/step - loss: 0.0391 - acc: 0.9847 - val_loss: 0.0432 - val_acc: 0.9832
Epoch 4/10
106381/106381 [==============================] - 126s 1ms/step - loss: 0.0363 - acc: 0.9854 - val_loss: 0.0440 - val_acc: 0.9837
Epoch 5/10
106381/106381 [==============================] - 125s 1ms/step - loss: 0.0334 - acc: 0.9867 - val_loss: 0.0446 - val_acc: 0.9827
Epoch 6/10
106381/106381 [==============================] - 125s 1ms/step - loss: 0.0307 - acc: 0.9876 - val_loss: 0.0456 - val_acc: 0.9833
Epoch 7/10
106381/106381 [==============================] - 126s 1ms/step - loss: 0.0285 - acc: 0.9885 - val_loss: 0.0463 - val_acc: 0.9824
Epoch 8/10
106381/106381 [==============================] - 126s 1ms/step - loss: 0.0261 - acc: 0.9895 - val_loss: 0.0484 - val_acc: 0.9827
Epoch 9/10
106381/106381 [=====

119678/119678 [==============================] - 135s 1ms/step - loss: 0.0236 - acc: 0.9903 - val_loss: 0.0516 - val_acc: 0.9822
Train on 119678 samples, validate on 39893 samples
Epoch 1/10
119678/119678 [==============================] - 135s 1ms/step - loss: 0.0564 - acc: 0.9798 - val_loss: 0.0442 - val_acc: 0.9826
Epoch 2/10
119678/119678 [==============================] - 135s 1ms/step - loss: 0.0430 - acc: 0.9833 - val_loss: 0.0414 - val_acc: 0.9838
Epoch 3/10
119678/119678 [==============================] - 135s 1ms/step - loss: 0.0395 - acc: 0.9845 - val_loss: 0.0413 - val_acc: 0.9835
Epoch 4/10
119678/119678 [==============================] - 135s 1ms/step - loss: 0.0365 - acc: 0.9856 - val_loss: 0.0421 - val_acc: 0.9833
Epoch 5/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0339 - acc: 0.9865 - val_loss: 0.0421 - val_acc: 0.9836
Epoch 6/10
119678/119678 [==============================] - 135s 1ms/step - loss: 0.0316 - acc: 0.9873 - val_loss: 0.043

Epoch 8/8
106380/106380 [==============================] - 132s 1ms/step - loss: 0.0395 - acc: 0.9843 - val_loss: 0.0412 - val_acc: 0.9835
Train on 106381 samples, validate on 53190 samples
Epoch 1/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0801 - acc: 0.9732 - val_loss: 0.0485 - val_acc: 0.9822
Epoch 2/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0501 - acc: 0.9812 - val_loss: 0.0491 - val_acc: 0.9809
Epoch 3/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0470 - acc: 0.9822 - val_loss: 0.0448 - val_acc: 0.9827
Epoch 4/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0453 - acc: 0.9827 - val_loss: 0.0434 - val_acc: 0.9833
Epoch 5/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0439 - acc: 0.9830 - val_loss: 0.0420 - val_acc: 0.9839
Epoch 6/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0422 - acc: 0.9835 - val_loss: 0

Epoch 2/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0495 - acc: 0.9816 - val_loss: 0.0469 - val_acc: 0.9821
Epoch 3/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0464 - acc: 0.9824 - val_loss: 0.0437 - val_acc: 0.9835
Epoch 4/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0447 - acc: 0.9828 - val_loss: 0.0432 - val_acc: 0.9837
Epoch 5/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0432 - acc: 0.9834 - val_loss: 0.0425 - val_acc: 0.9837
Epoch 6/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0422 - acc: 0.9837 - val_loss: 0.0421 - val_acc: 0.9837
Epoch 7/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0411 - acc: 0.9838 - val_loss: 0.0420 - val_acc: 0.9836
Epoch 8/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0405 - acc: 0.9841 - val_loss: 0.0418 - val_acc: 0.9837
Train on 119678 samples, va